# Working with the NPU driver

The NPU is another accelerator with its own set of tools and utilities to interact with. In this notebook we'll walk through how to use the utilities that get shipped with the XDNA driver to inspect our NPU status.

## Goals

* Get to know the XDNA driver
* Learn to use `xrt-smi` to check status
* Validate your device before moving on

## References

* [XDNA Driver Repo](https://github.com/amd/xdna-driver)
* [NPU Management Docs](https://ryzenai.docs.amd.com/en/latest/xrt_smi.html)

## Examining the Device

Much like the GPU where you have utilities like `rocm-smi` to view the status and resource usage of your GPU, when we work with the NPU we use `xrt-smi`. Like most bash utilities in Linux you can get an overview of what the program does by appending a `--help` flag.

In [1]:
!sudo /opt/xilinx/xrt/bin/xrt-smi examine

System Configuration
  OS Name              : Linux
  Release              : 6.14.0-32-generic
  Machine              : x86_64
  CPU Cores            : 32
  Memory               : 64068 MB
  Distribution         : Ubuntu 24.04.3 LTS
  GLIBC                : 2.39
  Model                : HP Z2 Mini G1a Workstation Desktop PC
  BIOS Vendor          : HP
  BIOS Version         : X53 Ver. 01.02.03

XRT
  Version              : 2.20.0
  Branch               : HEAD
  Hash                 : 021204355eeaa034ff69aae407ace2265adf047a
  Hash Date            : 2025-10-11 04:32:38
  amdxdna              : 2.20.0_20251007, 0ad5aa3765bb969135df1ae8ee26dae0bf65559a
  virtio-pci           : unknown, unknown
  NPU Firmware Version : 255.0.4.33

Device(s) Present
|BDF             |Name            |
|----------------|----------------|
|[0000:c4:00.1]  |NPU Strix Halo  |




In [2]:
!/opt/xilinx/xrt/bin/xrt-smi --help

XRT build version: 2.20.0
Build hash: 021204355eeaa034ff69aae407ace2265adf047a
Build date: 2025-10-11 04:32:38
Git branch: HEAD
PID: 263
UID: 1000
[Wed Oct 15 21:12:46 2025 GMT]
HOST: jupyter-roscon01
EXE: /opt/xilinx/xrt/bin/unwrapped/xrt-smi
[xrt-smi] ERROR: mmap(addr=0x7c92dc000000, len=67108864, prot=3, flags=8209, offset=4294967296) failed (err=-11): Resource temporarily unavailable


Most commonly you will want to use the `examine` command. This will give you a system overview as well as driver and firmware info -- crucial when submitting bug reports and debugging! The hashes correspond to commits the driver was built with - this is all open source and you can always update your driver to the bleeding edge by building from https://github.com/amd/xdna-driver.

In [3]:
!sudo /opt/xilinx/xrt/bin/xrt-smi examine

System Configuration
  OS Name              : Linux
  Release              : 6.14.0-32-generic
  Machine              : x86_64
  CPU Cores            : 32
  Memory               : 64068 MB
  Distribution         : Ubuntu 24.04.3 LTS
  GLIBC                : 2.39
  Model                : HP Z2 Mini G1a Workstation Desktop PC
  BIOS Vendor          : HP
  BIOS Version         : X53 Ver. 01.02.03

XRT
  Version              : 2.20.0
  Branch               : HEAD
  Hash                 : 021204355eeaa034ff69aae407ace2265adf047a
  Hash Date            : 2025-10-11 04:32:38
  amdxdna              : 2.20.0_20251007, 0ad5aa3765bb969135df1ae8ee26dae0bf65559a
  virtio-pci           : unknown, unknown
  NPU Firmware Version : 255.0.4.33

Device(s) Present
|BDF             |Name            |
|----------------|----------------|
|[0000:c4:00.1]  |NPU Strix Halo  |




We can scope down and report only the platform information about the NPU device. This is useful to know the power mode and how many columns are available.

In [4]:
!sudo /opt/xilinx/xrt/bin/xrt-smi examine --report platform


--------------------------------
[0000:c4:00.1] : NPU Strix Halo
--------------------------------
Platform
  Name                   : NPU Strix Halo 
  Power Mode             : Default 
  Total Columns          : 8 

Estimated Power          : N/A



## Reporting Hardware Contexts

This is a super important command because it will give you a populated list of running NPU applications. Sometimes when developing on the NPU or if you download an application that is meant to run on this platform you will want to know if it is **actually** running on the NPU. How can you tell?

**Hardware contexts** are spatial allocations on the NPU array for a given application. The XDNA driver and NPU firmware work together to provision a spatial partition on the NPU's array, dedicating a set of columns for the workload.

To examine these allocations we use the `--report aie-partitions` suboption.

In [5]:
!sudo /opt/xilinx/xrt/bin/xrt-smi examine --report aie-partitions


--------------------------------
[0000:c4:00.1] : NPU Strix Halo
--------------------------------
AIE Partitions
  No hardware contexts running on device



If nothing is running on the NPU this is the expected output. Additionally you can also use a handy all-in-one `--report all` suboption, to report everything! Again, knowing the system state is very useful for debugging.

In [6]:
!sudo /opt/xilinx/xrt/bin/xrt-smi examine --report all

System Configuration
  OS Name              : Linux
  Release              : 6.14.0-32-generic
  Machine              : x86_64
  CPU Cores            : 32
  Memory               : 64068 MB
  Distribution         : Ubuntu 24.04.3 LTS
  GLIBC                : 2.39
  Model                : HP Z2 Mini G1a Workstation Desktop PC
  BIOS Vendor          : HP
  BIOS Version         : X53 Ver. 01.02.03

XRT
  Version              : 2.20.0
  Branch               : HEAD
  Hash                 : 021204355eeaa034ff69aae407ace2265adf047a
  Hash Date            : 2025-10-11 04:32:38
  amdxdna              : 2.20.0_20251007, 0ad5aa3765bb969135df1ae8ee26dae0bf65559a
  virtio-pci           : unknown, unknown
  NPU Firmware Version : 255.0.4.33

Device(s) Present
|BDF             |Name            |
|----------------|----------------|
|[0000:c4:00.1]  |NPU Strix Halo  |



--------------------------------
[0000:c4:00.1] : NPU Strix Halo
--------------------------------
AIE Partitions
  No hardware context

## Validating device

OK now that we know that the NPU device exists and we can probe it for more info let's validate that the driver works as intended in our environment. We can use the `validate` subcommand for this. It will run a series of standard tests, and if everything passes we can be fairly confident that our device is functioning as intended.

In [7]:
!sudo /opt/xilinx/xrt/bin/xrt-smi validate --batch

Validate Device           : [0000:c4:00.1]
    Platform              : NPU Strix Halo
    Power Mode            : Performance
-------------------------------------------------------------------------------
Running Test: ..
Test 1 [0000:c4:00.1]     : gemm 
    Details               : TOPS: 51.0
    Test Status           : [PASSED]
-------------------------------------------------------------------------------
Running Test: ..
Test 2 [0000:c4:00.1]     : latency 
    Details               : Average latency: 57.0 us
    Test Status           : [PASSED]
-------------------------------------------------------------------------------
Running Test: .
Test 3 [0000:c4:00.1]     : throughput 
    Details               : Average throughput: 75588.0 op/s
    Test Status           : [PASSED]
-------------------------------------------------------------------------------
Validation completed. Please run the command '--verbose' option for more details


Excellent, your NPU device is operational and the driver is set up accordingly!

## Examine NPU Usage

Now that we have something that runs on the NPU - let's use what we learned to examine what a running application hardware context looks like.

We'll create a subprocess to run our validation tests. And while that is running let's examine the aie-partitions report.

In [10]:
import subprocess

# Run the command in the background
process = subprocess.Popen(
    ['sudo', '-E', '/opt/xilinx/xrt/bin/xrt-smi', 'validate'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print(f"Process started with PID: {process.pid}")

Process started with PID: 489


In [11]:
!sudo /opt/xilinx/xrt/bin/xrt-smi examine --report aie-partitions


--------------------------------
[0000:c4:00.1] : NPU Strix Halo
--------------------------------
AIE Partitions
  Total Memory Usage: N/A
  Partition Index   : 0
    Columns: [0, 1, 2, 3, 4, 5, 6, 7]
    HW Contexts:
      |PID                 |Ctx ID     |Submissions |Migrations  |Err  |Priority |
      |Process Name        |Status     |Completions |Suspensions |     |GOPS     |
      |Memory Usage        |Instr BO   |            |            |     |FPS      |
      |                    |           |            |            |     |Latency  |
      |====================|===========|============|============|=====|=========|
      |644902              |1          |102         |0           |0    |Normal   |
      |N/A                 |Active     |101         |0           |     |N/A      |
      |N/A                 |2476 KB    |            |            |     |N/A      |
      |                    |           |            |            |     |N/A      |
      |--------------------|------

Since the tests are utilizing the full array we see all 8 columns being used. Once we run our own applications, if we see this hw context table being populated we can be confident that the NPU is being utilized.

Next we will move to more practical programming exercises.

---
Copyright© 2025 AMD, Inc SPDX-License-Identifier: MIT